In [5]:
import numpy as np, pandas as pd, geopandas as gpd
import os
import matplotlib.pyplot as plt
import geopandas
import matplotlib.colors as colors

%load_ext nb_black
%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
##style graphs
from matplotlib import rcParams

rcParams["font.family"] = "Garamond"
rcParams["font.size"] = 14
rcParams["axes.spines.right"] = False
rcParams["axes.spines.top"] = False
rcParams["axes.spines.left"] = False
rcParams["axes.spines.bottom"] = False

rcParams["xtick.bottom"] = False
rcParams["xtick.labelbottom"] = False
rcParams["ytick.left"] = False
rcParams["ytick.labelleft"] = False

In [4]:
##load country boundaries
countries = gpd.read_file("data/GIS/TM_WORLD_BORDERS-0.3.shp")
countries = countries.to_crs("+proj=eqearth")
#countries.crs

##load world bank country data
country_df = pd.read_csv("data/country_data.csv", encoding="latin-1")
#country_df.head()

In [24]:
## load integrated world bank data file - see DevEmiss.ipynb for table creation code
wb_df = pd.read_csv("data/co2_gdp_wb.csv")

## remove zero and negative values
wb_df.loc[wb_df["CO2E.PC"] <= 0, "CO2E.PC"] = np.nan

## gdp per co2 per capita
wb_df["GDP.PCO2.PCAP"] = wb_df["GDP.PCAP"] / wb_df["CO2E.PC"]

## co2 per gdp per capita
wb_df["CO2.PGDP.PCAP"] = wb_df["CO2E.PC"] / wb_df["GDP.PCAP"]

# wb_df.columns

##select relevant columns
gdp_pco2_df = wb_df[
    [
        "Country Name",
        "Country Code",
        "year",
        "CO2E.PC",
        "GDP.PCAP",
        "GDP.PCO2.PCAP",
        "CO2.PGDP.PCAP",
    ]
]
# gdp_pco2_df.head()

<IPython.core.display.Javascript object>

In [25]:
# dict of columns labels
val_lab = {
    "CO2E.PC": "CO2 emissions (ton per capita)",
    "GDP.PCAP": "GDP (current US$ per capita)",
    "GDP.PCO2.PCAP": "GDP (current US$) per CO2 (ton) per capita",
    "CO2.PGDP.PCAP": "CO2 (ton) per GDP (current US$) per capita",
}

val_cmap = {
    "CO2E.PC": "coolwarm",
    "GDP.PCAP": "coolwarm_r",
    "GDP.PCO2.PCAP": "coolwarm_r",
    "CO2.PGDP.PCAP": "coolwarm",
}

# return value min and max
def vmin_vmax(df, col):
    vmin = df[col].replace(0, np.nan).min()
    vmax = df[col].max()

    return vmin, vmax


## clean table - remove years with unavailable data, replace 0 and negative values
def cleanTable(col):
    gdp_pco2_pivot = (
        gdp_pco2_df.replace(0, np.nan)
        .pivot(index="year", columns="Country Code", values=col)
        .dropna(how="all")
    )
    df = (
        gdp_pco2_pivot.transpose()
        .reset_index()
        .melt(id_vars="Country Code", value_name=col)
    )
    name_code = gdp_pco2_df[["Country Code", "Country Name"]]
    df = df.merge(name_code, on="Country Code").drop_duplicates().reset_index(drop=True)

    return df[["Country Name", "Country Code", "year"] + [col]]


## create world plot fig for col
def createWorldPlot(cntry_y, col, filelist, vmin, vmax):

    y = cntry_y["year"].unique().item()

    cmap = val_cmap[col]

    fig, ax = plt.subplots(1, 1, figsize=(7.5, 7.5), dpi=300)

    cntry_y.dropna().plot(
        column=col,
        ax=ax,
        cmap=cmap,
        alpha=0.9,
        norm=colors.LogNorm(vmin=vmin, vmax=vmax),
    )
    plt.axis("off")
    countries.plot(color="none", edgecolor="black", ax=ax, linewidth=0.1)

    sm = plt.cm.ScalarMappable(cmap=cmap, norm=colors.LogNorm(vmin=vmin, vmax=vmax))
    cbaxes = fig.add_axes([0.2, 0.285, 0.66, 0.03])
    cbar = fig.colorbar(sm, cax=cbaxes, orientation="horizontal", label=val_lab[col])
    fig.text(0.25, 0.4, y, weight="bold", size=18)

    path = "graphs/maps"
    if col not in os.listdir(path):
        os.mkdir("{}/{}".format(path, col))
    filename = "{}/{}/{}.png".format(path, col, y)
    filelist = filelist.append(filename)
    fig.savefig(filename, bbox_inches="tight")
    plt.close("all")

<IPython.core.display.Javascript object>

In [27]:
## co2 emissions/GDP/GDP p CO2 col
# for col in ["CO2E.PC", "GDP.PCAP", "GDP.CO2.PCAP", "CO2.PGDP.PCAP"]:
for col in ["GDP.PCAP"]:

    # clean table
    df = cleanTable(col)
    filelist = []

    ### create worldplot maps for col
    for year in df.year.sort_values().unique():
        vmin, vmax = vmin_vmax(df, col)
        y_df = df.loc[df.year == year].reset_index(drop=True)
        cntry_y = countries[["ISO3", "geometry"]].merge(
            y_df, left_on="ISO3", right_on="Country Code"
        )
        createWorldPlot(cntry_y, col, filelist, vmin, vmax)
    # print(filelist)

<IPython.core.display.Javascript object>

In [ ]:
## CREATE GIFs

# PIL AND imageio were low quality, use gifski (https://gif.ski) instead
# windows command line cd to graphs folder and run: 
#gifski -o gifs/co2e_pc.gif maps/CO2E.PC/*.png --fps 8
#gifski -o gifs/gdp_pc.gif maps/GDP.PCAP/*.png --fps 8

#gifski -o gifs/co2_pgdp_pc.gif maps/CO2.PGDP.PCAP/*.png --fps 4
#gifski -o gifs/gdp_pco2_pc.gif maps/GDP.PCO2.PCAP/*.png --fps 4



# from PIL import Image

# frames = []
# for filename in filelist:
#     im = Image.open(filename)
#     frames.append(im)

# frames[0].save(
#     "graphs/maps/{}.gif".format(col),
#     format="GIF",
#     append_images=frames[1:],
#     save_all=True,
#     duration=100,
#     loop=0,
# )


# import imageio

# images = []
# for filename in filelist:
#     images.append(imageio.imread(filename))
# imageio.mimsave("graphs/maps/{}.gif".format(col), images,  duration=0.1)